In [1]:
from biopandas.pdb import PandasPdb
from openbabel import openbabel, pybel
from pathlib import Path
import sys
import pandas as pd
from copy import copy
from tqdm import tqdm
import glob
import os



In [2]:
pdb_lip = dict()
with open('data/pdb_lipid.txt', 'r') as f:
    pdb_lip = {line.split()[0]: line.split()[1] for line in f.read().splitlines()}

In [3]:
base_path = Path('/Users/bouceond/phd/datasets/umol_predictions/')

for pdb in base_path.iterdir():
    if pdb.stem.startswith('.'):
        continue
    for file in glob.glob(str(pdb / '*.sdf')):
        os.remove(file)


for f in Path('tmp/').iterdir():
    os.remove(f)

In [ ]:

# ground_truth
for pdb in tqdm(base_path.iterdir(), total=len(list(base_path.iterdir()))):
    if pdb.stem.startswith('.'):
        continue
    # if pdb.stem.lower() != '7bwl':
    #     continue

    # ground truth
    aa = pdb
    pdb_code = pdb.stem.lower()
    ligand = pdb_lip[pdb_code]
    gt_path = pdb / 'output/gt_align.pdb'
    with open(gt_path, 'r') as f:
        lines = f.readlines()
    filtered_lines = list(filter(lambda x: x.startswith('HETATM') and (x[17:20].strip() == ligand), lines))

    atom_idxs = {int(line[6:11].strip()) for line in filtered_lines}
    insertions = {line[16].strip() for line in filtered_lines}
    # if len(insertions) > 1:
    #     # print(pdb_code)
    #     insertions = {""}
    connections = list(filter(lambda x: x.startswith('CONECT'), lines))
    new_connections = []
    for connection in connections:
        atoms = connection.split()[1:]
        out_connection = ['CONECT']
        out_connection.extend([atom for atom in atoms if int(atom) in atom_idxs])
        if len(out_connection) == 1:
            continue
        new_connection = ' '.join(out_connection) +'\n'
        new_connections.append(new_connection)
    # # print(connections)
    if len(insertions) == 1:
        new_path = f'tmp/{pdb_code}_mol_true.pdb'
        with open(new_path, 'w') as f:
            for line in filtered_lines:
                f.write(line)
            for connection in new_connections:
                f.write(connection)
        # antechamber -i new.pdb -fi pdb -o new.mol2 -fo mol2 -at sybyl
        mol2_path = f'tmp/{pdb_code}_mol_true.mol2'
        cmd = f'antechamber -i {new_path} -fi pdb -o {mol2_path} -fo mol2 -at sybyl -dr no -j 5'
        !{cmd}

        supplier = pybel.readfile('mol2', mol2_path)
        # print(supplier)
        mol = next(supplier)
        mol.removeh()
        new_path = f'{pdb}/mol_true.sdf'
        mol.write('sdf', new_path, overwrite=True)

    else:
        print(pdb_code)
        for insertion in insertions:
            new_path = f'tmp/{pdb_code}_mol_true_{insertion}.pdb'

            if insertion == "":
                continue
            filtered_lines_copy = copy(filtered_lines)
            filtered_lines_copy = list(filter(lambda x: x[16].strip() in [insertion, ""], filtered_lines_copy))
            atom_idxs = {int(line[6:11].strip()) for line in filtered_lines_copy}
            new_connections = []
            for connection in connections:
                atoms = connection.split()[1:]
                out_connection = ['CONECT']
                out_connection.extend([atom for atom in atoms if int(atom) in atom_idxs])
                # out_connection.extend([atom for atom in atoms])
                if len(out_connection) == 1:
                    continue
                new_connection = ' '.join(out_connection) +'\n'
                new_connections.append(new_connection)

                with open(new_path, 'w') as f:
                    for line in filtered_lines_copy:
                        f.write(line)
                    for connection in new_connections:
                        f.write(connection)

            mol2_path = f'tmp/{pdb_code}_mol_true_{insertion}.mol2'
            cmd = f'antechamber -i {new_path} -fi pdb -o {mol2_path} -fo mol2 -at sybyl -dr no -j 5'
            !{cmd}

            supplier = pybel.readfile('mol2', mol2_path)
            # print(supplier)
            mol = next(supplier)
            mol.removeh()
            new_path = f'{pdb}/mol_true_{insertion}.sdf'
            mol.write('sdf', new_path, overwrite=True)



    # predicted
    predicted_path = pdb / 'output/pred_align.pdb'

    with open(predicted_path, 'r') as f:
        lines = f.readlines()
    filtered_lines = list(filter(lambda x: x.startswith('HETATM') and (x[17:20].strip() == 'UNK'), lines))

    atom_idxs = {int(line[6:11].strip()) for line in filtered_lines}
    insertions = {line[16].strip() for line in filtered_lines}
    connections = list(filter(lambda x: x.startswith('CONECT'), lines))
    new_connections = []
    for connection in connections:
        atoms = connection.split()[1:]
        out_connection = ['CONECT']
        out_connection.extend([atom for atom in atoms if int(atom) in atom_idxs])
        if len(out_connection) == 1:
            continue
        new_connection = ' '.join(out_connection) +'\n'
        new_connections.append(new_connection)

    new_path = f'tmp/{pdb_code}_mol_pred.pdb'
    with open(new_path, 'w') as f:
        for line in filtered_lines:
            f.write(line)
        for connection in new_connections:
            f.write(connection)
    print(f'{new_path=} saved')
    mol2_path = f'tmp/{pdb_code}_mol_pred.mol2'
    cmd = f'antechamber -i {new_path} -fi pdb -o {mol2_path} -fo mol2 -at sybyl -dr no -j 5'
    !{cmd}

    supplier = pybel.readfile('mol2', mol2_path)
    # print(supplier)
    mol = next(supplier)
    mol.removeh()
    new_path = f'{pdb}/mol_pred.sdf'
    mol.write('sdf', new_path, overwrite=True)



In [5]:
0/0

!antechamber -i tmp/7w8f_mol_true.pdb -fi pdb -o tmp/7w8f_mol_true.mol2 -fo mol2 -at sybyl -j 5 -s 0



ZeroDivisionError: division by zero

In [ ]:
supplier = pybel.readfile('mol2', 'tmp/4qf7_mol_true.mol2')
# print(supplier)
mol = next(supplier)
mol.removeh()
new_path = f'{pdb}/mol_true.sdf'
mol.write('sdf', 'out.sdf', overwrite=True)

In [ ]:
s = """/Users/bouceond/phd/datasets/umol_predictions/1KQW/mol_cond.pdb,/Users/bouceond/phd/datasets/umol_predictions/1KQW/mol_true_A.sdf,/Users/bouceond/phd/datasets/umol_predictions/1KQW/mol_pred.sdf
/Users/bouceond/phd/datasets/umol_predictions/1KQW/mol_cond.pdb,/Users/bouceond/phd/datasets/umol_predictions/1KQW/mol_true_B.sdf,/Users/bouceond/phd/datasets/umol_predictions/1KQW/mol_pred.sdf""".split('\n')
for a in s:
    print(*a.split(','))

/Users/bouceond/phd/datasets/umol_predictions/1KQW/mol_cond.pdb /Users/bouceond/phd/datasets/umol_predictions/1KQW/mol_true_A.sdf /Users/bouceond/phd/datasets/umol_predictions/1KQW/mol_pred.sdf
/Users/bouceond/phd/datasets/umol_predictions/1KQW/mol_cond.pdb /Users/bouceond/phd/datasets/umol_predictions/1KQW/mol_true_B.sdf /Users/bouceond/phd/datasets/umol_predictions/1KQW/mol_pred.sdf


In [ ]:
# prediction


for pdb in base_path.iterdir():
    if pdb.stem.startswith('.'):
        continue

    pdb_code = pdb.stem.lower()
    ligand = pdb_lip[pdb_code]
    gt_path = pdb / 'output/pred_align.pdb'
    ppdb = PandasPdb().read_pdb(str(gt_path))
    # print(ppdb)

    hetatm_df = ppdb.df['HETATM'].copy()
    hetatm_df = hetatm_df.loc[hetatm_df['residue_name'] == 'UNK']
    name = f'{pdb_code}_pred.pdb'
    ppdb.to_pdb(f'tmp/{name}', records=['HETATM'])

    mol = next(pybel.readfile('pdb', f'tmp/{name}'))
    name = f'mol_pred.sdf'
    new_path = str(pdb / name)
    # print(new_path, file=sys.stderr)
    mol.write('sdf', new_path, overwrite=True)


In [ ]:
0/0
df = PandasPdb().read_pdb(str(aa / 'output/gt_align.pdb')).df
df['HETATM']


In [ ]:
p = '/Users/bouceond/phd/datasets/umol_predictions/6BC4/output/pred_align.pdb'
print(p)

In [ ]:
mol = next(pybel.readfile('pdb', f'tmp/6bc4_pred.pdb'))
mol

In [ ]:
data = []
for p in base_path.iterdir():
    if p.stem.startswith('.'):
        continue
    trues = [f for f in p.iterdir() if f.stem.startswith('mol_true') and f.suffix == '.sdf']

    mol_cond = p / 'mol_cond.pdb'
    mol_pred = p / 'mol_pred.sdf'

    for true in trues:
        # print(pred_file)
        data.append([mol_cond, true, mol_pred])
pd.DataFrame(data, columns=['path_cond', 'path_true', 'path_pred']).to_csv('data/umol_lipid.csv', index=False)

In [ ]:
0/0

In [ ]:
obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats('pdb', 'sdf')
mol = openbabel.OBMol()
mol = next(pybel.readfile('pdb', 'test.pdb'))
mol.write('sdf', f'test.sdf', overwrite=True)

In [ ]:
!antechamber


Welcome to antechamber 22.0: molecular input file processor.

Usage: antechamber -i     input file name
                   -fi    input file format
                   -o     output file name
                   -fo    output file format
                   -c     charge method
                   -cf    charge file name
                   -nc    net molecular charge (int)
                   -a     additional file name
                   -fa    additional file format
                   -ao    additional file operation
                          crd   : only read in coordinate
                          crg   : only read in charge
                          radius: only read in radius
                          name  : only read in atom name
                          type  : only read in atom type
                          bond  : only read in bond type 
                   -m     multiplicity (2S+1), default is 1
                   -rn    residue name, overrides input file, default is MOL
    

In [ ]:
from openbabel import pybel

In [ ]:
?pybel.readfile